In [2]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [13]:
import os
os.chdir(r"C:\Users\coach\myfiles\postdoc\Fire\code\Burn_Area_Mapping\src")
os.getcwd()

'C:\\Users\\coach\\myfiles\\postdoc\\Fire\\code\\Burn_Area_Mapping\\src'

In [20]:
from logger import logging
from data_preprocessing import preProcessyCollection
from data_preprocessing import preProcessXCollection

In [21]:
from exception import customException

In [22]:
%load_ext autoreload
%autoreload 2

In [9]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
# Filter countries by geometry
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
geometry = ee.Geometry.Point([24.06353794842853, -29.732969740562062])
sa = countries.filterBounds(geometry)
sa_geo = sa.geometry()

# Define start and end dates
startDate = ee.Date.fromYMD(2018, 7, 4)
endDate = ee.Date.fromYMD(2022, 8, 15)

10


In [19]:
from geeml.utils import eeprint
eeprint(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(sa_geo).filterDate(startDate,endDate).limit(3).map(lambda i: cloudMask(sensor = 'Sentinel-2', image =i)))

In [ ]:
X = preProcessXCollection(collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'), region = sa_geo, startDate = startDate, endDate = endDate)

In [16]:
# from src.exception import customException
import sys

y = preProcessyCollection(collection = ee.ImageCollection("ESA/CCI/FireCCI/5_1"), region = sa_geo, startDate = startDate, endDate = endDate)
X = preProcessXCollection(collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED"), region = sa_geo, startDate = startDate, endDate = endDate)

In [20]:
fire = ee.ImageCollection("ESA/CCI/FireCCI/5_1")
fire = fire.first()
from geeml.utils import eeprint
eeprint(fire.bandNames())